In [7]:
def calc_outlet_xP(data,HFC, IL, k21,k12,verbose=False):
    '''Inputs
            Data - csv file for x,T,P data HFC-IL
            HFC  - string with name of HFC (i.e."R125")
            IL   - string with name of IL (i.e. "emimTf2N")
            k21  - binary parameter IL-HFC
            k12  - binary parameter HFC-IL
            
            '''
    # create zeros arrays to be filled with x1 and P1
    HFC_x = np.zeros((len(data)))
    P = np.zeros((len(data)))
    
    # data labels
    x_IL= "x_"+IL
    x_HFC= "x_"+HFC
    
    #print
    print(x_IL)
    print(x_HFC)
    
    # model constraints
    m.fs.liq = Param(mutable=True,default=0.040)
    m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, IL] == m.fs.liq)

    for i in range(len(data)):
        m.fs.liq = data[x_IL].iloc[i]
        m.fs.F101.inlet.flow_mol.fix(1) 
        m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[i]))
        m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[i]))
        m.fs.F101.inlet.mole_frac_comp[0, HFC].fix(float(data[x_HFC].iloc[i])+0.1)
        m.fs.F101.inlet.mole_frac_comp[0, IL].fix(float(1-(data[x_HFC].iloc[i]+0.1)))
        m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[i]))
        m.fs.properties.PR_kappa[IL, HFC].fix(k21) # (-0.20093)
        m.fs.properties.PR_kappa[HFC, IL].fix(k12) # (-0.05619)

        if verbose:
            DOF_final = degrees_of_freedom(m)
            print("The final DOF is {0}".format(DOF_final))

        # solver
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6}
        status = solver.solve(m, tee = False)
    #     m.fs.F101.report()
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            HFC_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,HFC]) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
        else:
            print('Infeasible.')
            
    return(HFC_x,P)